# WK Doc-classifier Model 
[Preparatory work]

In [2]:
#for pretty print to all vars
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all" #default="last_expr"

#to_display_any_htmls
from IPython.core.display import display, HTML

In [3]:
#Change dir to read dataset files from zip's extract
import os
os.chdir('C:/Users/mihir.parikh/Downloads/AoI-275/')
files=list()
categories=[fl for fl in os.listdir('.') if os.path.isdir(fl)]
for folder in categories:
        files+=[folder+'/'+p for p in os.listdir(folder)]
print(categories," Total files read:",len(files))

['AoI-275', 'MC-275', 'Others-275']  Total files read: 825


In [4]:
#preprocess file's text_contents before laoding to pandas dataframe
import re
dataF={'content':[],'catg':[]}
newfiles=[]
#adding to dataFrame after preprocessing:
#    [remove 1st few not null lines, remove files of less than 10 chars, merge concurrent spaces lines]
skip1stLines=2
for fi in files:
    with open(fi,encoding="utf8") as f:
        #print("for file",fi)
        PREPROC=f.read()
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        while PREPROC.count('\n '):PREPROC=PREPROC.replace('\n ','\n')
        while PREPROC.count(' \n'):PREPROC=PREPROC.replace(' \n','\n')
        while PREPROC.count('\n\n'):PREPROC=PREPROC.replace('\n\n','\n')
        PREPROC=' '.join(PREPROC.split('\n')[0 if fi.startswith('Others') else skip1stLines:])
        #PREPROC=re.sub(r"\d+(\.\d*)?", r"",PREPROC) #remove all numbers (digits)
        PREPROC=re.sub(r"[(](\S*?)[)]", r"",PREPROC) #remove all b/w (brackets)
        PREPROC=re.sub(r"[^a-zA-Z ]", r"",PREPROC) #remove all non alpha (brackets)
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        if len(PREPROC)<10:continue
        dataF['content'].append(PREPROC)
        dataF['catg'].append(fi.split('-')[0])
        newfiles.append(fi)
skippedfiles=list(set(files)-set(newfiles))
print(len(skippedfiles),"files ignored. New Dataset has",len(newfiles),"files.")
##check skip files of some catg
#sorted(list(filter((lambda st:st.startswith('MC')),skippedfiles)))

43 files ignored. New Dataset has 782 files.


In [5]:
#load to pandas dataframe and have a peek on each catg
import pandas as pd
df = pd.DataFrame.from_dict(dataF)
print(df['catg'].value_counts())
for k in df['catg'].value_counts().keys():display(HTML(df[(df['catg']==k)].head(5).to_html()))

Others    275
MC        265
AoI       242
Name: catg, dtype: int64


,catg,content
507,Others,Hines Global REIT II Inc INQUIRY LETTER April...
508,Others,PNM Resources Inc INQUIRY LETTER January VIA ...
509,Others,Exhibit CERTIFICATION PURSUANT TO USC SECTION...
510,Others,Exhibit FOREVERGREEN WORLDWIDE CORPORATION CE...
511,Others,Exhibit CERTIFICATION PURSUANT TO SECTION OF ...


,catg,content
242,MC,FIRST AMENDMENT TO ADMINISTRATIVE SERVICES AGR...
243,MC,AMENDMENT NO TO ADMINISTRATIVE SERVICES AGREEM...
244,MC,AMENDMENT NO TO ADMINISTRATIVE SERVICES AGREEM...
245,MC,FIFTH AMENDMENT TO LOAN AND SECURITY AGREEMENT...
246,MC,A LDEYRA T HERAPEUTICS I NC C HANGE IN C ONTRO...


,catg,content
0,AoI,CERTIFICATE OF AMENDMENT TO ARTICLES OF INCORP...
1,AoI,CERTIFICATE OF AMENDMENT OF CERTIFICATE OF INC...
2,AoI,AMENDED AND RESTATED BYLAWS OF DIPEXIUM PHARMA...
3,AoI,BY LAWS of INTER PARFUMS INC changed from Jean...
4,AoI,REGISTRE DU COMMERCE DU CANTON DE FRIBOURG EXT...


In [6]:
#partial import of sklearn. will full load sub-modules as needed.
import sklearn

#split data for training and testing
from sklearn.model_selection import train_test_split
X=df['content']
Y=df['catg']
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.402,random_state=402)
print(X_train.shape,X_test.shape, Y_train.shape, Y_test.shape)
#now data is ready

(467,) (315,) (467,) (315,)


### Multinomial NAIVE BAYES

In [7]:
#pipelining
from sklearn.pipeline import Pipeline

#naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf_nb = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.01))
                    ])

#fit_transforms of data to pipeline
fit_nb=text_clf_nb.fit(X_train,Y_train)

#testing model
print("--> Accuracy")
print("TRAIN",text_clf_nb.score(X_train,Y_train))
print("TEST",text_clf_nb.score(X_test,Y_test))

#misClassifications
print("--> Train MisClassification")
TRpredicted_nb = text_clf_nb.predict(X_train)
misClTr_nb=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_nb,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_nb)):
    if TRpredicted_nb[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_nb[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_nb.append(i)
print(len(misClTr_nb)," misses in TrainData")
print("--> Test MisClassification")
TSpredicted_nb = text_clf_nb.predict(X_test)
misClTs_nb=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_nb,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_nb)):
    if TSpredicted_nb[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_nb[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_nb.append(i)
print(len(misClTs_nb)," misses in TestData")


--> Accuracy
TRAIN 0.9957173447537473
TEST 0.9555555555555556
--> Train MisClassification


Train_Predicted,AoI,MC,Others,All
True,,,,
AoI,153,0,0,153
MC,0,142,0,142
Others,0,2,170,172
All,153,144,170,467


X_Train.values[ 245 ] Predicted MC Actual Others File:[errI=544 Others-275/new 132.txt ]
X_Train.values[ 285 ] Predicted MC Actual Others File:[errI=543 Others-275/new 131.txt ]
2  misses in TrainData
--> Test MisClassification


Test_Predicted,AoI,MC,Others,All
True,,,,
AoI,88,0,1,89
MC,2,120,1,123
Others,2,8,93,103
All,92,128,95,315


X_test.values[ 20 ] Predicted MC Actual Others File:[errI=625 Others-275/new 205.txt ]
X_test.values[ 57 ] Predicted Others Actual AoI File:[errI=105 AoI-275/0001193794-03-000005-5.txt ]
X_test.values[ 68 ] Predicted MC Actual Others File:[errI=545 Others-275/new 133.txt ]
X_test.values[ 118 ] Predicted MC Actual Others File:[errI=580 Others-275/new 165.txt ]
X_test.values[ 132 ] Predicted MC Actual Others File:[errI=537 Others-275/new 126.txt ]
X_test.values[ 145 ] Predicted MC Actual Others File:[errI=578 Others-275/new 163.txt ]
X_test.values[ 152 ] Predicted Others Actual MC File:[errI=305 MC-275/0001213900-17-003750-2.txt ]
X_test.values[ 180 ] Predicted MC Actual Others File:[errI=547 Others-275/new 135.txt ]
X_test.values[ 187 ] Predicted MC Actual Others File:[errI=559 Others-275/new 146.txt ]
X_test.values[ 201 ] Predicted AoI Actual Others File:[errI=620 Others-275/new 200.txt ]
X_test.values[ 208 ] Predicted AoI Actual MC File:[errI=500 MC-275/0001553350-17-000313-4.txt ]
X_

In [8]:
#study misclassified doc errI [by filename or errI_index]
#errI=newfiles.index('MC-275/0001553350-17-000313-4.txt')
errI=543
print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

543 TRAINdata Others-275/new 131.txt  NEITHER THIS SECURITY NOR THE SECURITIES FOR WHICH THIS SECURITY IS EXERCISABLE HAVE BEEN REGISTERED WITH THE SECURITIES AND EXCHANGE COMMISSION OR THE SECURITIES COMMISSION OF ANY STATE IN RELIANCE UPON AN EXEMPTION FROM REGISTRATION UNDER THE SECURITIES ACT OF AS AMENDED THE SECURITIES ACT AND ACCORDINGLY MAY NOT BE OFFERED OR SOLD EXCEPT PURSUANT TO AN EFFECTIVE REGISTRATION STATEMENT UNDER THE SECURITIES ACT OR PURSUANT TO AN AVAILABLE EXEMPTION FROM OR IN A TRANSACTION NOT SUBJECT TO THE REGISTRATION REQUIREMENTS OF THE SECURITIES ACT AND IN ACCORDANCE WITH APPLICABLE STATE SECURITIES LAWS THIS SECURITY AND THE SECURITIES ISSUABLE UPON EXERCISE OF THIS SECURITY MAY BE PLEDGED IN CONNECTION WITH A BONA FIDE MARGIN ACCOUNT OR OTHER LOAN SECURED BY SUCH SECURITIES COMMON STOCK PURCHASE WARRANT AIR INDUSTRIES GROUP Warrant Shares Initial Exercise Date April THIS COMMON STOCK PURCHASE WARRANT the Warrant certifies that for value received Taglich Br

### SVM SVC [C-Support Vector Classification]

In [9]:
#pipelining
from sklearn.pipeline import Pipeline

#svm_svc
from sklearn.svm import SVC
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto'))
                    ])

#fit_transforms of data to pipeline
fit_svm=text_clf_svm.fit(X_train,Y_train)

#testing model
print("--> Accuracy")
print("TRAIN",text_clf_svm.score(X_train,Y_train))
print("TEST",text_clf_svm.score(X_test,Y_test))

#misClassifications
print("--> Train MisClassification")
TRpredicted_svm = text_clf_svm.predict(X_train)
misClTr_svm=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_svm,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_svm)):
    if TRpredicted_svm[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_svm[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_svm.append(i)
print(len(misClTr_svm)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_svm = text_clf_svm.predict(X_test)
misClTs_svm=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_svm,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_svm)):
    if TSpredicted_svm[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_svm[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_svm.append(i)
print(len(misClTs_svm)," misses in TestData")


--> Accuracy
TRAIN 0.9978586723768736
TEST 0.9492063492063492
--> Train MisClassification


Train_Predicted,AoI,MC,Others,All
True,,,,
AoI,153,0,0,153
MC,0,142,0,142
Others,0,1,171,172
All,153,143,171,467


X_Train.values[ 245 ] Predicted MC Actual Others File:[errI=544 Others-275/new 132.txt ]
1  misses in TrainData
--> Test MisClassification


Test_Predicted,AoI,MC,Others,All
True,,,,
AoI,86,0,3,89
MC,0,116,7,123
Others,0,6,97,103
All,86,122,107,315


X_test.values[ 20 ] Predicted MC Actual Others File:[errI=625 Others-275/new 205.txt ]
X_test.values[ 33 ] Predicted Others Actual MC File:[errI=361 MC-275/0001387131-17-001719-2.txt ]
X_test.values[ 48 ] Predicted Others Actual MC File:[errI=302 MC-275/0001213900-17-003160-6.txt ]
X_test.values[ 57 ] Predicted Others Actual AoI File:[errI=105 AoI-275/0001193794-03-000005-5.txt ]
X_test.values[ 118 ] Predicted MC Actual Others File:[errI=580 Others-275/new 165.txt ]
X_test.values[ 132 ] Predicted MC Actual Others File:[errI=537 Others-275/new 126.txt ]
X_test.values[ 134 ] Predicted Others Actual AoI File:[errI=22 AoI-275/0001171843-15-001395-3.txt ]
X_test.values[ 145 ] Predicted MC Actual Others File:[errI=578 Others-275/new 163.txt ]
X_test.values[ 152 ] Predicted Others Actual MC File:[errI=305 MC-275/0001213900-17-003750-2.txt ]
X_test.values[ 180 ] Predicted MC Actual Others File:[errI=547 Others-275/new 135.txt ]
X_test.values[ 187 ] Predicted MC Actual Others File:[errI=559 Oth

In [10]:
#study misclassified doc errI [by filename or errI_index]
errI=newfiles.index('Others-275/new 126.txt')
#errI=496-1
print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

537 TESTdata Others-275/new 126.txt  Exhibit Adopted by the Board of Directors on December Adopted by the Stockholders on themaven inc STOCK INCENTIVE PLAN Purpose The purpose of this theMaven Inc Stock Incentive Plan the Plan is to advance the interests of theMaven Inc the Company and its stockholders by enabling the Company and its subsidiaries to attract and retain qualified individuals through opportunities for equity participation in the Company and to reward those individuals who contribute to the Companys achievement of its economic objectives Definitions The following terms will have the meanings set forth below unless the context clearly otherwise requires Board means the Board of Directors of the Company Broker Exercise Notice means a written notice pursuant to which a Participant upon exercise of an Option irrevocably instructs a broker or dealer to sell a sufficient number of shares or loan a sufficient amount of money to pay all or a portion of the exercise price of the Op

### Decision Tree

In [11]:
#pipelining
from sklearn.pipeline import Pipeline

#DT
from sklearn.tree import DecisionTreeClassifier
text_clf_dt = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf',DecisionTreeClassifier(
                         criterion='gini', splitter='best', max_depth=None, 
                         min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                         max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                         min_impurity_split=None, class_weight=None, presort=False))
                    ])

#fit_transforms of data to pipeline
fit_dt=text_clf_dt.fit(X_train,Y_train)

#testing model
print("--> Accuracy")
print("TRAIN",text_clf_dt.score(X_train,Y_train))
print("TEST",text_clf_dt.score(X_test,Y_test))

#misClassifications
print("--> Train MisClassification")
TRpredicted_dt = text_clf_dt.predict(X_train)
misClTr_dt=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_dt,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_dt)):
    if TRpredicted_dt[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_dt[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_dt.append(i)
print(len(misClTr_dt)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_dt = text_clf_dt.predict(X_test)
misClTs_dt=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_dt,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_dt)):
    if TSpredicted_dt[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_dt[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_dt.append(i)
print(len(misClTs_dt)," misses in TestData")


--> Accuracy
TRAIN 1.0
TEST 0.834920634920635
--> Train MisClassification


Train_Predicted,AoI,MC,Others,All
True,,,,
AoI,153,0,0,153
MC,0,142,0,142
Others,0,0,172,172
All,153,142,172,467


0  misses in TrainData
--> Test MisClassification


Test_Predicted,AoI,MC,Others,All
True,,,,
AoI,81,6,2,89
MC,6,91,26,123
Others,4,8,91,103
All,91,105,119,315


X_test.values[ 9 ] Predicted MC Actual Others File:[errI=719 Others-275/new 42.txt ]
X_test.values[ 14 ] Predicted Others Actual MC File:[errI=255 MC-275/0001193125-17-106850-6.txt ]
X_test.values[ 28 ] Predicted Others Actual MC File:[errI=271 MC-275/0001193125-17-123414-6.txt ]
X_test.values[ 35 ] Predicted Others Actual MC File:[errI=478 MC-275/0001493152-17-003856-4.txt ]
X_test.values[ 37 ] Predicted AoI Actual MC File:[errI=289 MC-275/0001213900-17-003042-4.txt ]
X_test.values[ 57 ] Predicted Others Actual AoI File:[errI=105 AoI-275/0001193794-03-000005-5.txt ]
X_test.values[ 60 ] Predicted MC Actual Others File:[errI=637 Others-275/new 216.txt ]
X_test.values[ 61 ] Predicted Others Actual MC File:[errI=432 MC-275/0001477932-17-001489-3.txt ]
X_test.values[ 63 ] Predicted Others Actual MC File:[errI=320 MC-275/0001223389-17-000017-4.txt ]
X_test.values[ 65 ] Predicted AoI Actual MC File:[errI=431 MC-275/0001477932-17-001489-2.txt ]
X_test.values[ 91 ] Predicted Others Actual MC F

In [12]:
#study misclassified doc errI [by filename or errI_index]
errI=newfiles.index('MC-275/0001553350-17-000313-4.txt')
errI=495
print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

495 TRAINdata MC-275/0001515971-17-000057-2.txt MODIFICATION TO THE AGREEMENT This Modification to the GEMSTONE PROCESSING AGREEMENT The Agreement made and entered into by and between QUALITECH INC HEREAFTER QTI and INTERNATIONAL ISOTOPES INC hereafter INIS WITNESSETH WHEREAS INIS has requested an adjustment to the Processing Charge and implementation of an annual inflationary index to make further adjustments to that Processing Charge and WHEREAS QTI is willing to allow such adjustments NOW THEREFORE in consideration of the above the parties agree to modify the original Agreement dated June as follows Effective January the Processing Charge will be increased to per carat On March I and each year thereafter the Processing Charge shall be adjusted by an amount equal to the annual percentage change in the Services Special Aggregate index listed in Table of data provided by the US Bureau of Labor Statistics Historical CPI All other provisions of the Agreement remain in full force and effe

### Linear Standard Grad. Descent

In [13]:
#pipelining
from sklearn.pipeline import Pipeline

#sgd
from sklearn.linear_model import SGDClassifier
text_clf_sgd = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf',SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None))
                    ])

#fit_transforms of data to pipeline
fit_sgd=text_clf_sgd.fit(X_train,Y_train)

#testing model
print("--> Accuracy")
print("TRAIN",text_clf_sgd.score(X_train,Y_train))
print("TEST",text_clf_sgd.score(X_test,Y_test))

#misClassifications
print("--> Train MisClassification")
TRpredicted_sgd = text_clf_sgd.predict(X_train)
misClTr_sgd=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_train,TRpredicted_sgd,rownames=["True"],colnames=["Train_Predicted"],margins=True).to_html()))
for i in range(len(TRpredicted_sgd)):
    if TRpredicted_sgd[i]!=Y_train.iloc[i]:
        print("X_Train.values[",i,"] Predicted",TRpredicted_sgd[i],"Actual",Y_train.iloc[i],
              "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
        misClTr_sgd.append(i)
print(len(misClTr_sgd)," misses in TrainData")

print("--> Test MisClassification")
TSpredicted_sgd = text_clf_sgd.predict(X_test)
misClTs_sgd=list()
#sklearn.metrics.confusion_matrix
display(HTML(pd.crosstab(Y_test,TSpredicted_sgd,rownames=["True"],colnames=["Test_Predicted"],margins=True).to_html()))
for i in range(len(TSpredicted_sgd)):
    if TSpredicted_sgd[i]!=Y_test.iloc[i]:
        print("X_test.values[",i,"] Predicted",TSpredicted_sgd[i],"Actual",Y_test.iloc[i],
              "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
        misClTs_sgd.append(i)
print(len(misClTs_sgd)," misses in TestData")


--> Accuracy
TRAIN 0.9978586723768736
TEST 0.9555555555555556
--> Train MisClassification


Train_Predicted,AoI,MC,Others,All
True,,,,
AoI,153,0,0,153
MC,0,142,0,142
Others,0,1,171,172
All,153,143,171,467


X_Train.values[ 245 ] Predicted MC Actual Others File:[errI=544 Others-275/new 132.txt ]
1  misses in TrainData
--> Test MisClassification


Test_Predicted,AoI,MC,Others,All
True,,,,
AoI,87,0,2,89
MC,0,118,5,123
Others,0,7,96,103
All,87,125,103,315


X_test.values[ 20 ] Predicted MC Actual Others File:[errI=625 Others-275/new 205.txt ]
X_test.values[ 33 ] Predicted Others Actual MC File:[errI=361 MC-275/0001387131-17-001719-2.txt ]
X_test.values[ 48 ] Predicted Others Actual MC File:[errI=302 MC-275/0001213900-17-003160-6.txt ]
X_test.values[ 57 ] Predicted Others Actual AoI File:[errI=105 AoI-275/0001193794-03-000005-5.txt ]
X_test.values[ 68 ] Predicted MC Actual Others File:[errI=545 Others-275/new 133.txt ]
X_test.values[ 118 ] Predicted MC Actual Others File:[errI=580 Others-275/new 165.txt ]
X_test.values[ 132 ] Predicted MC Actual Others File:[errI=537 Others-275/new 126.txt ]
X_test.values[ 134 ] Predicted Others Actual AoI File:[errI=22 AoI-275/0001171843-15-001395-3.txt ]
X_test.values[ 145 ] Predicted MC Actual Others File:[errI=578 Others-275/new 163.txt ]
X_test.values[ 152 ] Predicted Others Actual MC File:[errI=305 MC-275/0001213900-17-003750-2.txt ]
X_test.values[ 180 ] Predicted MC Actual Others File:[errI=547 Othe

In [15]:
#study misclassified doc errI [by filename or errI_index]
errI=newfiles.index('Others-275/new 206.txt')
#errI=495
print(errI,"TESTdata" if errI in X_test else "TRAINdata",newfiles[errI],X_test.get(errI,0) or X_train.get(errI,0))
#os.system("np++ "+newfiles[errI])

626 TRAINdata Others-275/new 206.txt  AGREEMENT AS TO VOTING AGREEMENT as to voting effective November by and among WILLIAM C MOOG JANE B MOOG RICHARD A AUBRECHT NANCY MOOG AUBRECHT CONSTANCE MOOG SILLIMAN JEANNE M MOOG DOUGLAS B MOOG SUSAN L MOOG JAMES R SILLIMAN Jr and MICHAEL K SILLIMAN all individually referred to as previously indicated or as Holder and sometimes collectively referred to as Holders and ALBERT K HILL WITNESSETH THAT WHEREAS A WCM was the founder and motivating force in the organization and success of Moog Inc a New York corporation the Company and B WCM JBM and RAA are currently Directors and WCM is the Chairman of the Board President and Chief Executive Officer and RAA is also an Executive Officer of the Company and C Holders in their own right andor in various capacities such as custodians trustees trust beneficiaries and custodial beneficiaries are record andor beneficial owners of a substantial number of shares of Class A Common Stock and Class B Common Stock e

## Tinkering with models

In [18]:
text_clf_nb.predict_proba([X_train[626]]),text_clf_nb.classes_

(array([[9.61320746e-12, 4.81030500e-11, 1.00000000e+00]]),
 array(['AoI', 'MC', 'Others'], dtype='<U6'))

In [22]:
rsTr=text_clf_nb.predict_proba(X_train)

In [50]:
sum([sum(x) for x in rs[:100]])-100

9.947598300641403e-14

In [47]:
#trs=sum([max(x) for x in rsTr])/len(rsTr)
p=1
trs=(sum([(max(x))**p for x in rsTr])/len(rsTr))**(1/p)
trs

0.9996213359923677

In [38]:
rs=text_clf_nb.predict_proba(X_test)

In [60]:
for i in misClTs_nb:
    if max(rs[i])<trs:#,"less by",-max(rs[i])+trs
        print(i,"chale",rs[i],"pr far from true=",max(rs[i])-rs[text_clf_nb.classes_.tolist().index(Y_test.iloc[i])])
for i in misClTs_nb:
    if max(rs[i])>trs:
        print(i,"Locha",rs[i],"extra by",max(rs[i])-trs,"pr far from true=",)
#chale=>no catg has enough confidence on them
#Locha=>wrong catg has confidence on them by <-->


57 chale [0.11332278 0.05022813 0.83644909] pr far from true= [-0.16355091  0.83644909  0.83644909]
68 chale [1.91335522e-05 9.51611310e-01 4.83695561e-02] pr far from true= [-0.04838869  0.95161131  0.95161131]
152 chale [1.34868641e-09 4.57441860e-01 5.42558138e-01] pr far from true= [ 0.54255814  0.54255813 -0.45744186]
187 chale [4.70825585e-05 9.82678851e-01 1.72740663e-02] pr far from true= [-0.01732115  0.98267885  0.98267885]
201 chale [0.92186769 0.02100059 0.05713172] pr far from true= [-0.07813231  0.92186769  0.92186769]
221 chale [4.48036081e-04 9.93033252e-01 6.51871195e-03] pr far from true= [-0.00696675  0.99303325  0.99303325]
225 chale [0.49836349 0.01687536 0.48476114] pr far from true= [-0.50163651  0.49836349  0.49836349]
262 chale [0.69456395 0.27571229 0.02972376] pr far from true= [ 0.69456395  0.69456394 -0.30543605]
20 Locha [7.46892910e-05 9.99917294e-01 8.01712301e-06] extra by 0.00029595759356615403 pr far from true=
118 Locha [6.98211735e-08 9.99999709e-01

In [58]:
trueCatg=text_clf_nb.classes_.tolist().index(Y_test.iloc[misClTs_nb[3]])
